<a id='Оглавление'></a>
### [Цель проекта](#Цель_проекта) 
### [Описание данных](#Описание_данных) 
### [Первичный осмотр данных](#Первичный_осмотр_данных) 
### [Предобработка данных](#Предобработка_данных) 
 #### - [Cтолбец children](#Проверим_столбец_children) 
 #### - [Cтолбец dob_years](#Проверим_столбец_dob_years)
 #### - [Cтолбец education](#Проверим_столбец_education)
 #### - [Обработка дубликатов](#дубликаты)
 #### - [Столбец family_status](#столбец_family_status)
 #### - [Столбец income_type](#столбец_income_type)
 #### - [Столбец_days_employed](#столбец_days_employed)
### [Обработка пропусков](#Обработка_пропусков) 
### [Лемматизация](#лемматизация) 
### [Категоризация данных](#категоризация_данных) 
 #### -  [Категоризации по цели_кредита](#категоризации_по_цели_кредита) 
 #### -  [Категоризации по семейному_положению](#категоризации_по_семейному_положению) 
 #### -  [Категоризации по наличию_детей](#категоризации_по_наличию_детей) 
 #### -  [Категоризация по доходу](#категоризация_по_доходу) 
### [Общий вывод](#Общий_вывод) 


<a id='Цель_проекта'></a>
### Цель проекта: исследование надёжности заёмщиков
[Оглавление](#Оглавление)

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<a id='Описание_данных'></a>
### Описание данных
[Оглавление](#Оглавление)
   - children — количество детей в семье
   - days_employed — общий трудовой стаж в днях
   - dob_years — возраст клиента в годах
   - education — уровень образования клиента
   - education_id — идентификатор уровня образования
   - family_status — семейное положение
   - family_status_id — идентификатор семейного положения
   - gender — пол клиента
   - income_type — тип занятости
   - debt — имел ли задолженность по возврату кредитов
   - total_income — ежемесячный доход
   - purpose — цель получения кредита

<a id='Первичный_осмотр_данных'></a>
### Первичный осмотр данных. 
[Оглавление](#Оглавление)

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Вывод по общей информации:
- days_employed (общий трудовой стаж в днях) имеет тип данных float64, хотя значения ожидаются целочисленные
- в столбцах days_employed и total_income пропуски

In [2]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


#### Вывод по знакомству с данными:
- данные в столбце days_employed (общий трудовой стаж в днях) преимущественно отрицательные
- в столбце days_employed есть неправдоподобные значения (без учета знака), которые искажают средний показатель
- кол-во детей от -1 до 20
- минимальный возраст 0


<a id='Предобработка_данных'></a>
### Предобработка данных
[Оглавление](#Оглавление)

#### Проверим данные на наличие дубикатов

In [5]:
#data.duplicated().sum()

In [6]:
#data = data.drop_duplicates().reset_index(drop = True)

In [7]:
#data.duplicated().sum()

<a id='Проверим_столбец_children'></a>
#### Проверим столбец children
[Оглавление](#Оглавление)

In [8]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Наибольшее кол-во детей оказалось 20, без промежуточных значений это является ошибкой. 

In [9]:
data['children'] = data['children'].replace(20, 2)

In [10]:
# -1 ребенка превратим в 1 по аналогичной причине
data['children'] = data['children'].replace(-1, 1)

In [11]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

<a id='Проверим_столбец_dob_years'></a>
#### Проверим столбец dob_years
[Оглавление](#Оглавление)

In [12]:
data[data['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

Все некорректные значения оказались нулями, заменим их медианными значениями возраста по типу занятости (income_type)

In [13]:
# Как оказалось это столбец нам не пригодится
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.median())))

In [14]:
data[data['dob_years'] < 18]['dob_years'].count()

0

<a id='Проверим_столбец_education'></a>
#### Проверим столбец education
[Оглавление](#Оглавление)

In [15]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [16]:
data['education'] = data['education'].str.lower()

In [17]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

<a id='дубликаты'></a>
#### Обработаем дубликаты
[Оглавление](#Оглавление)

In [18]:
# Было 54
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates().reset_index(drop = True)

In [20]:
data.duplicated().sum()

0

<a id='столбец_family_status'></a>
#### Проверим столбец family_status
[Оглавление](#Оглавление)

In [21]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

<a id='столбец_income_type'></a>
#### Проверим столбец income_type
[Оглавление](#Оглавление)

In [22]:
data['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

<a id='столбец_days_employed'></a>
#### Проверим столбец days_employed
[Оглавление](#Оглавление)

In [23]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [24]:
days_employed = data.groupby('income_type').agg({'days_employed':['count', lambda x: sum(x>0)]})
days_employed

days_employed           
                        count <lambda_0>
income_type                             
безработный                 2        2.0
в декрете                   1        0.0
госслужащий              1312        0.0
компаньон                4577        0.0
пенсионер                3443     3443.0
предприниматель             1        0.0
сотрудник               10014        0.0
студент                     1        0.0

Все аномальные значения находят в категории пенсионеры, для этой категории стаж не имеет значения. Можем преобразовывать отрицательные значения и тип данных столбца.

In [25]:
data['days_employed'] = data['days_employed'].abs()

In [26]:
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

<a id='Обработка_пропусков'></a>
### Обработка пропусков
[Оглавление](#Оглавление)

In [27]:
data.isnull().sum()

children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
dtype: int64

In [28]:

data[data['days_employed'].isnull()].equals(data[data['total_income'].isnull()])

True

#### Вывод по пропускам:
- строки с пропущенными данными полностью совпали
- появление пропусков при выгрузке выглядит маловероятным
- т.к. выявлено наличие связи между столбцами, пропуски могут возникать на этапе ввода данных (без стажа не учитывается зарплата и без зарплаты не принимается стаж). 


In [29]:
pivot_table_fillna = pd.pivot_table(data, index = ['education', 'income_type'], values = ['days_employed', 'total_income'], aggfunc = np.median)
pivot_table_fillna

days_employed   total_income
education           income_type                                  
высшее              безработный      395302.838654  202722.511368
                    госслужащий        2531.034209  172511.107016
                    компаньон          1454.659104  201785.400018
                    пенсионер        366158.526428  144240.768611
                    предприниматель     520.848083  499163.144947
                    сотрудник          1556.457266  165640.744634
                    студент             578.751554   98201.625314
начальное           госслужащий        2787.767403  148339.290825
                    компаньон          1151.634460  136798.905143
                    пенсионер        360264.985350  102598.653164
                    сотрудник          1197.176853  125994.910603
неоконченное высшее госслужащий        1885.183639  160592.345303
                    компаньон          1017.888120  179867.152890
                    пенсионер        372250.501660  120136.896353
                    сотрудник          1197.676879  151308.937846
среднее             безработный      337524.466835   59956.991984
                    в декрете          3296.759962   53829.130729
                    госслужащий        2857.770974  136652.970357
                    компаньон          1670.809068  159070.690289
                    пенсионер        365025.338867  114842.854099
                    сотрудник          1613.581420  136555.108821
ученая степень      госслужащий        5968.075884  111392.231107
                    пенсионер        356930.517546  177088.845999
                    сотрудник          2351.431934  198570.757322

Задаем функции заполения пропусков на основе данных из сводной таблицы

In [30]:
def fillna_days(row):
    return pivot_table_fillna.loc[row['education']].loc[row['income_type']]['days_employed']

In [31]:
def fillna_income(row):
    return pivot_table_fillna.loc[row['education']].loc[row['income_type']]['total_income']

In [32]:
data.loc[data['days_employed'].isnull(), 'days_employed'] = data.apply(fillna_days, axis = 1)

In [33]:
#data.loc[data['total_income'].isnull(), 'total_income'] = data.apply(fillna_days, axis = 1)

In [34]:
data.loc[data['total_income'].isnull(), 'total_income'] = data.apply(fillna_income, axis = 1)

In [35]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены, теперь можно преобразовать тип данных столбцов days_employed и total_income

In [36]:
data['days_employed'] = data['days_employed'].astype(int)

In [37]:
data['total_income'] = data['total_income'].astype(int)

In [38]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [40]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [41]:
#До любой обработки данных кол-во пенсионеров 3856
data[data['income_type'] == 'пенсионер']['income_type'].count()

3829

In [42]:
#До любой обработки данных кол-во пенсионеров со стажем > 0 == 3443
data[(data['income_type'] == 'пенсионер') & (data['days_employed'] > 0)]['income_type'].count()

3829

In [43]:
#До любой обработки данных кол-во пенсионеров со стажем меньше или равно нулю отстутвуют, т.е. пропущены
data[(data['income_type'] == 'пенсионер') & (data['days_employed'] <= 0)]['income_type'].count()

0

In [44]:
#До любой обработки данных возраст 75% пенсионеров больше 56 лет
data[data['income_type'] == 'пенсионер']['dob_years'].describe()

count    3829.000000
mean       59.361974
std         6.323446
min        22.000000
25%        56.000000
50%        60.000000
75%        64.000000
max        74.000000
Name: dob_years, dtype: float64

In [45]:
# До любой обработки данных кол-во не пенсионеров старше 56 лет 1154
data[(data['income_type'] != 'пенсионер') & (data['dob_years'] > 56)]['dob_years'].count()

1149

In [46]:
# До любой обработки данных стаж не пенсионеров старше 56 лет: 
# mean = -3667.583099506628 // median = -2493.117902101178 // mean для старше 64 лет -4010.47835666443

data[(data['income_type'] != 'пенсионер') & (data['dob_years'] > 64)]['days_employed'].mean() #mean()

# Вывод: менять стаж у пенсионеров на 10 лет не сильно корректней, чем оставить по 900 изначальных

# В реальной ситуации стоит уточнить у источника данных как он получил такие значения, 
 


3779.2868217054265

<a id='лемматизация'></a>
### Лемматизация
[Оглавление](#Оглавление)

In [47]:
import pprint
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Ознакомимся с общими формулировками целей кредита

In [48]:
unique_purpose = data['purpose'].unique()
unique_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [49]:
string = '; '.join(unique_purpose)
pprint.pprint(string)

('покупка жилья; приобретение автомобиля; дополнительное образование; сыграть '
 'свадьбу; операции с жильем; образование; на проведение свадьбы; покупка '
 'жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; '
 'покупка жилой недвижимости; строительство собственной недвижимости; '
 'недвижимость; строительство недвижимости; на покупку подержанного '
 'автомобиля; на покупку своего автомобиля; операции с коммерческой '
 'недвижимостью; строительство жилой недвижимости; жилье; операции со своей '
 'недвижимостью; автомобили; заняться образованием; сделка с подержанным '
 'автомобилем; получение образования; автомобиль; свадьба; получение '
 'дополнительного образования; покупка своего жилья; операции с недвижимостью; '
 'получение высшего образования; свой автомобиль; сделка с автомобилем; '
 'профильное образование; высшее образование; покупка жилья для сдачи; на '
 'покупку автомобиля; ремонт жилью; заняться высшим образованием')


In [50]:
lemmas = m.lemmatize(string)
pprint.pprint(Counter(lemmas))

Counter({' ': 59,
         '; ': 37,
         'покупка': 10,
         'недвижимость': 10,
         'автомобиль': 9,
         'образование': 9,
         'жилье': 7,
         'с': 5,
         'операция': 4,
         'на': 4,
         'свой': 4,
         'свадьба': 3,
         'строительство': 3,
         'получение': 3,
         'высокий': 3,
         'дополнительный': 2,
         'для': 2,
         'коммерческий': 2,
         'жилой': 2,
         'подержать': 2,
         'заниматься': 2,
         'сделка': 2,
         'приобретение': 1,
         'сыграть': 1,
         'проведение': 1,
         'семья': 1,
         'собственный': 1,
         'со': 1,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})


Можно выделить следующие типы целей кредита:
   - недвижимость
   - автомобиль
   - жилье
   - образование
   - свадьба

<a id='категоризация_данных'></a>
### Категоризация данных
[Оглавление](#Оглавление) 

<a id='категоризации_по_цели_кредита'></a>
#### Зададим функцию категоризации по цели кредита.
[Оглавление](#Оглавление)

In [51]:
purpose_category = ['недвижимость', 'автомобиль', 'жилье', 'образование', 'свадьба']

In [52]:
def categories_purpose(text):
    purpose = m.lemmatize(text)
    for word in purpose:
        for category in purpose_category:
            if category in purpose:
                return category
    

In [53]:
data['purpose_category'] = data['purpose'].apply(categories_purpose)

In [54]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [55]:
data['debt2'] = data['debt']
pivot_purpose = pd.pivot_table(data, index = 'purpose_category', columns = 'debt2', values = 'debt', aggfunc = 'count')
pivot_purpose['%'] = pivot_purpose[1] / (pivot_purpose[1] + pivot_purpose[0]) * 100
pivot_purpose.sort_values('%', ascending = False)

debt2,0,1,%
purpose_category,,,
автомобиль,3903,403,9.359034
образование,3643,370,9.220035
свадьба,2138,186,8.003442
недвижимость,5877,474,7.463392
жилье,4152,308,6.905830


#### Вывод:

Самыми рискованымми категориями являются образование и автомобили

<a id='категоризации_по_семейному_положению'></a>
#### Зададим функцию категоризации по семейному положению.
[Оглавление](#Оглавление)

In [56]:
def categorize_family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'

In [57]:
data['family_status_category'] = data['family_status'].apply(categorize_family_status)

In [58]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt2,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,0,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,0,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,0,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,не в браке


In [59]:
pd.pivot_table(data, index= 'family_status_category', values = 'debt')

,debt
family_status_category,
в браке,0.075452
не в браке,0.088865


In [60]:
#data.groupby('family_status')['debt'].agg(['count', 'mean']).sort_values('mean', ascending = False)

In [61]:
pivot_family = pd.pivot_table(data, index = 'family_status', columns = 'debt2', values = 'debt', aggfunc = 'count')
pivot_family['%'] = pivot_family[1] / (pivot_family[1] + pivot_family[0]) * 100
pivot_family.sort_values('%', ascending = False)

debt2,0,1,%
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


#### Вывод:
Люди в браке реже сталкиваются с проблемами при возврате кредита. 

<a id='категоризации_по_наличию_детей'></a>
#### Зададим функцию категоризации по наличию детей.
[Оглавление](#Оглавление)

In [62]:
def children_category(number):
    if number == 0:
        return 'нет детей'
    elif number > 2:
        return 'многодетная семья'
    else:
        return 'есть дети'

In [63]:
data['children_category'] = data['children'].apply(children_category)

In [64]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt2,family_status_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,0,в браке,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,0,в браке,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0,в браке,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,0,в браке,многодетная семья
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,не в браке,нет детей


In [65]:
pd.pivot_table(data, index= 'children_category', values = 'debt').sort_values('debt', ascending = False)

,debt
children_category,
есть дети,0.092654
многодетная семья,0.081579
нет детей,0.075438


#### Вывод:

Без детей кредиты возвращаются стабильнее. Самое интересно, что многодетные семьи справляются с выплатами лучше семей в которых 1-2 ребенка.

<a id='категоризация_по_доходу'></a>
#### Зададим функцию категоризации по доходу.
[Оглавление](#Оглавление)

In [66]:
data['total_income'].describe()

count    2.145400e+04
mean     1.654530e+05
std      9.828866e+04
min      2.066700e+04
25%      1.075155e+05
50%      1.437075e+05
75%      1.983070e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [67]:
def income_category(total_income):
    if total_income <= 107515:
            return 'Низкий уровень дохода'
    elif total_income <= 143707:
            return 'Средний уровень дохода'
    elif total_income < 198307:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'

In [68]:
data['income_category'] = data['total_income'].apply(income_category)

In [69]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt2,family_status_category,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,0,в браке,есть дети,Сверхвысокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,0,в браке,есть дети,Средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0,в браке,нет детей,Высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,0,в браке,многодетная семья,Сверхвысокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,не в браке,нет детей,Высокий уровень дохода


In [70]:
pd.pivot_table(data, index= 'income_category', values = 'debt').sort_values('debt', ascending = False)

,debt
income_category,
Средний уровень дохода,0.090434
Высокий уровень дохода,0.085027
Низкий уровень дохода,0.079605
Сверхвысокий уровень дохода,0.069538


#### Вывод:
Средний и высокий уровень дохода не гарантирует своевременных выплат.

<a id='Общий_вывод'></a>
### Общий вывод
[Оглавление](#Оглавление)

Результат анализа представлен в виде таблицы. К средней категории риска были отнесены параметры, имеющие равные или близкие к 8% показатели невозврата. В категорию высокого и низкого риска были распределены параметры каждой категории на основании max и min показателя невозврата, относительно среднего (8%).

In [71]:
results = {'Наличие детей':['1-2 ребенка', 'многодетные', 'без детей'], 'Цель кредита':['автомобиль', 'свадьба', 'жилье'], 
           'Уровень дохода':['средний', 'низкий', 'сверхвысокий'], 
           'Семейное положение':['не женат / не замужем', 'женат / замужем', 'вдовец / вдова'],} 

final = pd.DataFrame(results, index =['Высокий риск', 'Средний риск', 'Низкий риск']) 

final 

,Наличие детей,Цель кредита,Уровень дохода,Семейное положение
Высокий риск,1-2 ребенка,автомобиль,средний,не женат / не замужем
Средний риск,многодетные,свадьба,низкий,женат / замужем
Низкий риск,без детей,жилье,сверхвысокий,вдовец / вдова


Самыми благонадежными является клиенты в браке без детей, оформляющие кредит на жилье/недвижимость.  
Кредит одиноким людям с детьми на автомобиль/образование является наиболее рискованным для данной выборки.

Oтвет на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?": есть  
Oтвет на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?": есть  
Oтвет на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?":есть  
Oтвет на вопрос: "Как разные цели кредита влияют на его возврат в срок?": все категории не связанные с недвижимостью повышают риск возникновения задолжности по ним